In [ ]:
# Import live code changes in
%load_ext autoreload
%autoreload 2

from pathlib import Path
import os
import pandas as pd
import rasterio
from rasterstats import zonal_stats
from tqdm import tqdm
import numpy as np
import geopandas as gpd

import warnings
warnings.filterwarnings("ignore")

#### Set filepaths and provide data info

In [10]:
root = Path.cwd().parent # find project root
urban_area_adapted = 23 # towns and cities (see GHS-DUC documentation)
basin_path = os.path.join(root, 'outputs', 'boundaries', 'analysis_basins.gpkg')
risk_map_path = os.path.join(root, 'outputs', 'flood', 'risk', 'maps')
# risk_map_path = os.path.join(root, 'outputs', 'flood', 'sensitivity', 'maps', 'sd1_low') # for sensitivity analysis
output_path = os.path.join(root, 'outputs', 'flood', 'risk', 'basins')
# output_path = os.path.join(root, 'outputs', 'flood', 'sensitivity', 'basins', 'sd1_low') # for sensitivity analysis
adaptation_mask_path = os.path.join(root, 'outputs', 'flood', 'adaptation', f'urban_mask_{urban_area_adapted}.tif') # We want to mask out cities for adaptation... 
raster_info_list = [
    {"Sector": "Public", "RP": 10, "file": os.path.join(risk_map_path, 'JRC_10_pub_cap_damages.tif')},
    {"Sector": "Public", "RP": 20, "file": os.path.join(risk_map_path, 'JRC_20_pub_cap_damages.tif')},
    {"Sector": "Public", "RP": 50, "file": os.path.join(risk_map_path, 'JRC_50_pub_cap_damages.tif')},
    {"Sector": "Public", "RP": 75, "file": os.path.join(risk_map_path, 'JRC_75_pub_cap_damages.tif')},
    {"Sector": "Public", "RP": 100, "file": os.path.join(risk_map_path, 'JRC_100_pub_cap_damages.tif')},
    {"Sector": "Public", "RP": 200, "file": os.path.join(risk_map_path, 'JRC_200_pub_cap_damages.tif')},
    {"Sector": "Public", "RP": 500, "file": os.path.join(risk_map_path, 'JRC_500_pub_cap_damages.tif')},
    {"Sector": "Private", "RP": 10, "file": os.path.join(risk_map_path, 'JRC_10_priv_cap_damages.tif')},
    {"Sector": "Private", "RP": 20, "file": os.path.join(risk_map_path, 'JRC_20_priv_cap_damages.tif')},
    {"Sector": "Private", "RP": 50, "file": os.path.join(risk_map_path, 'JRC_50_priv_cap_damages.tif')},
    {"Sector": "Private", "RP": 75, "file": os.path.join(risk_map_path, 'JRC_75_priv_cap_damages.tif')},
    {"Sector": "Private", "RP": 100, "file": os.path.join(risk_map_path, 'JRC_100_priv_cap_damages.tif')},
    {"Sector": "Private", "RP": 200, "file": os.path.join(risk_map_path, 'JRC_200_priv_cap_damages.tif')},
    {"Sector": "Private", "RP": 500, "file": os.path.join(risk_map_path, 'JRC_500_priv_cap_damages.tif')},
    {"Sector": "Service", "RP": 10, "file": os.path.join(risk_map_path, 'JRC_10_ser_gva_losses.tif')},
    {"Sector": "Service", "RP": 20, "file": os.path.join(risk_map_path, 'JRC_20_ser_gva_losses.tif')},
    {"Sector": "Service", "RP": 50, "file": os.path.join(risk_map_path, 'JRC_50_ser_gva_losses.tif')},
    {"Sector": "Service", "RP": 75, "file": os.path.join(risk_map_path, 'JRC_75_ser_gva_losses.tif')},
    {"Sector": "Service", "RP": 100, "file": os.path.join(risk_map_path, 'JRC_100_ser_gva_losses.tif')},
    {"Sector": "Service", "RP": 200, "file": os.path.join(risk_map_path, 'JRC_200_ser_gva_losses.tif')},
    {"Sector": "Service", "RP": 500, "file": os.path.join(risk_map_path, 'JRC_500_ser_gva_losses.tif')},
    {"Sector": "Manufacturing", "RP": 10, "file": os.path.join(risk_map_path, 'JRC_10_man_gva_losses.tif')},
    {"Sector": "Manufacturing", "RP": 20, "file": os.path.join(risk_map_path, 'JRC_20_man_gva_losses.tif')},
    {"Sector": "Manufacturing", "RP": 50, "file": os.path.join(risk_map_path, 'JRC_50_man_gva_losses.tif')},
    {"Sector": "Manufacturing", "RP": 75, "file": os.path.join(risk_map_path, 'JRC_75_man_gva_losses.tif')},
    {"Sector": "Manufacturing", "RP": 100, "file": os.path.join(risk_map_path, 'JRC_100_man_gva_losses.tif')},
    {"Sector": "Manufacturing", "RP": 200, "file": os.path.join(risk_map_path, 'JRC_200_man_gva_losses.tif')},
    {"Sector": "Manufacturing", "RP": 500, "file": os.path.join(risk_map_path, 'JRC_500_man_gva_losses.tif')},
    {"Sector": "Agriculture", "RP": 10, "file": os.path.join(risk_map_path, 'JRC_10_agr_gva_losses.tif')},
    {"Sector": "Agriculture", "RP": 20, "file": os.path.join(risk_map_path, 'JRC_20_agr_gva_losses.tif')},
    {"Sector": "Agriculture", "RP": 50, "file": os.path.join(risk_map_path, 'JRC_50_agr_gva_losses.tif')},
    {"Sector": "Agriculture", "RP": 75, "file": os.path.join(risk_map_path, 'JRC_75_agr_gva_losses.tif')},
    {"Sector": "Agriculture", "RP": 100, "file": os.path.join(risk_map_path, 'JRC_100_agr_gva_losses.tif')},
    {"Sector": "Agriculture", "RP": 200, "file": os.path.join(risk_map_path, 'JRC_200_agr_gva_losses.tif')},
    {"Sector": "Agriculture", "RP": 500, "file": os.path.join(risk_map_path, 'JRC_500_agr_gva_losses.tif')},
]

In [11]:
# Load basins
basin_df = gpd.read_file(basin_path)
# Initializ an empty DataFrame to store zonal statistics
results_df = pd.DataFrame()

#### Run overlay analysis 

In [12]:
# Read mask outside loop
with rasterio.open(adaptation_mask_path) as msrc:
    mask = msrc.read(1)
    mask_transform = msrc.transform
    
for raster_info in tqdm(raster_info_list, desc="Running overlay analysis"):
    # Load raster data
    with rasterio.open(raster_info['file']) as src:
        flood = src.read(1)
        transform = src.transform

    # Exclude masked cells (to represent adapted regions)
    adapted_flood = np.where(mask==1, 0, flood)

    # Perform flooded and adapted zonal sum
    zs = zonal_stats(
        basin_df,
        flood,
        affine=transform,
        stats='sum',
        geojson_out=True
    )
    zs_adapted = zonal_stats(
        basin_df,
        adapted_flood,
        affine=transform,
        stats='sum',
        geojson_out=True
    )
    
    # Prepare a DataFrame from the zonal statistics
    temp_df = pd.DataFrame({
        "FID": [feat['id'] for feat in zs],
        "GID_1": [feat['properties']['flpr_gid_1'] for feat in zs],
        "NAME": [feat['properties']['NAME'] for feat in zs],
        "HB_L6": [feat['properties']['HYBAS_ID_06'] for feat in zs],
        "Pr_L": [feat['properties']['MerL_Riv'] for feat in zs],
        "damages": [feat['properties']['sum'] for feat in zs],
        'adapted_damages': [feat['properties']['sum'] for feat in zs_adapted]
    })
    
    # Add raster information to the DataFrame
    temp_df["RP"] = raster_info['RP']
    temp_df["Sector"] = raster_info['Sector']
    
    # Remove NaNs from damage column
    temp_df['damages'] = temp_df['damages'].fillna(0)
    temp_df['adapted_damages'] = temp_df['adapted_damages'].fillna(0)

    # Concatenate teh temporary DataFrame with the results DataFrame
    results_df = pd.concat([results_df, temp_df], ignore_index=True)

# Save results
if not os.path.exists(output_path):
    os.makedirs(output_path)
results_df.to_csv(os.path.join(output_path, 'risk_basins.csv'))

Running overlay analysis: 100%|████████████████████████████████████████████████████████| 35/35 [11:58<00:00, 20.51s/it]


PermissionError: [Errno 13] Permission denied: 'E:\\Projects\\sovereign-risk\\outputs\\flood\\risk\\basins\\risk_basins.csv'

In [13]:
results_df.to_csv(os.path.join(output_path, 'risk_basins.csv'))